In [9]:
import json
import copy
from pathlib import Path
from tqdm import tqdm  # 引入进度条，让你知道跑到了哪里

# --- 导入 BFCL 核心组件 ---
from bfcl_eval.eval_checker.multi_turn_eval.multi_turn_utils import execute_multi_turn_func_call
from bfcl_eval.eval_checker.multi_turn_eval.multi_turn_checker import multi_turn_checker

# --- 1. 健壮的数据加载器 ---
def smart_load_data(file_path):
    path = Path(file_path)
    if not path.exists():
        print(f"❌ 错误：找不到文件 {file_path}")
        return None

    with open(path, 'r', encoding='utf-8') as f:
        content = f.read().strip()

    try:
        data = json.loads(content)
        if isinstance(data, list):
            return data
    except json.JSONDecodeError:
        pass 

    try:
        data = []
        lines = content.splitlines()
        for i, line in enumerate(lines):
            line = line.strip()
            if not line: continue 
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                pass
        if len(data) > 0:
            return data
    except Exception:
        pass

    print(f"❌ 错误：无法解析文件数据。")
    return None

# --- 2. 模拟 LLM (作弊模式：直接返回答案) ---
def mock_llm_inference(history, current_turn_ground_truth):
    # 直接返回标准答案，模拟一个完美模型
    return current_turn_ground_truth

# --- 3. ReAct Agent (增加 verbose 参数控制日志) ---
# --- 3. ReAct Agent (修复版) ---
def run_react_agent(test_entry, ground_truth_turns, verbose=False):
    test_id = test_entry["id"]
    initial_config = test_entry["initial_config"]
    involved_classes = test_entry["involved_classes"]
    turns = test_entry["question"]
    
    all_turns_decoded_results = []
    history = [] 
    
    # 1. 初始化环境
    _, involved_instances = execute_multi_turn_func_call(
        func_call_list=[], 
        initial_config=initial_config,
        involved_classes=involved_classes,
        model_name="demo_runner",
        test_entry_id=test_id
    )

    # 2. 多轮循环
    for turn_idx, user_turn in enumerate(turns):
        user_msg = user_turn[0]
        history.append(user_msg)
        
        if verbose:
            print(f"👤 Turn {turn_idx}: {user_msg['content']}")
        
        # [修复点] 安全检查：防止答案长度小于问题长度
        if turn_idx < len(ground_truth_turns):
            current_ground_truth = ground_truth_turns[turn_idx]
        else:
            # 如果没有对应的答案，通常意味着这一轮不需要调用工具，或者数据有问题
            # 这里我们默认不做操作，或者打个日志
            if verbose: print(f"⚠️ Warning: Turn {turn_idx} 没有对应的 Ground Truth，跳过工具调用。")
            current_ground_truth = [] # 假定为空

        current_turn_steps_decoded = []
        
        # 3. 模拟推理
        func_call_strs = mock_llm_inference(history, current_ground_truth)
        
        step_decoded = []
        for call_str in func_call_strs:
            step_decoded.append(call_str)
        current_turn_steps_decoded.append(step_decoded)
        
        # 4. 执行工具
        if func_call_strs:
            if verbose: print(f"⚡ 执行: {func_call_strs}")
            execution_results, _ = execute_multi_turn_func_call(
                func_call_list=func_call_strs,
                initial_config=initial_config,
                involved_classes=involved_classes,
                model_name="demo_runner",
                test_entry_id=test_id
            )
            for exec_res in execution_results:
                history.append({"role": "tool", "content": str(exec_res)})

        all_turns_decoded_results.append(current_turn_steps_decoded)

    return all_turns_decoded_results

# --- 4. 主入口 (全量运行版) ---
def main():
    # ⚠️ 请确认路径正确
    prompt_file = "/dfs/data/work/gorilla/berkeley-function-call-leaderboard/bfcl_eval/data/BFCL_v3_multi_turn_base.json"
    answer_file = "/dfs/data/work/gorilla/berkeley-function-call-leaderboard/bfcl_eval/data/possible_answer/BFCL_v3_multi_turn_base.json"
    
    print(f"📂 正在加载全量数据...")
    all_prompts = smart_load_data(prompt_file)
    all_answers = smart_load_data(answer_file)
    
    if not all_prompts or not all_answers:
        return

    # 建立索引加速查找
    answer_map = {item['id']: item for item in all_answers}
    
    # 统计变量
    total_cases = len(all_prompts)
    passed_count = 0
    failed_cases = []

    print(f"🚀 开始全量评测 (共 {total_cases} 条数据)...")
    print("-" * 50)

    # 使用 tqdm 显示进度条
    for test_entry in tqdm(all_prompts, desc="Running Mock Evaluation"):
        test_id = test_entry['id']
        
        # 1. 获取答案
        if test_id not in answer_map:
            print(f"\n⚠️ 跳过: 找不到 ID {test_id} 的答案")
            failed_cases.append(test_id)
            continue
        
        ground_truth_entry = answer_map[test_id]

        try:
            # 2. 运行 Agent (verbose=False 关闭详细日志，只跑逻辑)
            model_decoded_output = run_react_agent(
                test_entry, 
                ground_truth_entry["ground_truth"], 
                verbose=False
            )
            
            # 3. 打分
            score_result = multi_turn_checker(
                model_decoded_output, 
                ground_truth_entry["ground_truth"], 
                test_entry, 
                "multi_turn_base", 
                "demo_runner"
            )
            
            if score_result["valid"]:
                passed_count += 1
            else:
                # 记录失败详情
                failed_cases.append({
                    "id": test_id,
                    "error": score_result.get("error")
                })
                
        except Exception as e:
            print(f"\n❌ 运行时崩溃 ID {test_id}: {e}")
            failed_cases.append({"id": test_id, "error": str(e)})

    # --- 最终报告 ---
    print("\n" + "="*50)
    print(f"📊 评测完成报告")
    print(f"Total: {total_cases}")
    print(f"Passed: {passed_count}")
    print(f"Failed: {len(failed_cases)}")
    
    if total_cases > 0:
        accuracy = (passed_count / total_cases) * 100
        print(f"Accuracy: {accuracy:.2f}%")
    
    if failed_cases:
        print("\n❌ 失败用例列表:")
        for fail in failed_cases:
            print(fail)
    else:
        print("\n🎉 完美！Mock 模型全部通过！Pipeline 验证无误。")

if __name__ == "__main__":
    main()

📂 正在加载全量数据...
🚀 开始全量评测 (共 200 条数据)...
--------------------------------------------------


Running Mock Evaluation: 100%|█████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 558.74it/s]


❌ 运行时崩溃 ID multi_turn_base_154: list index out of range

📊 评测完成报告
Total: 200
Passed: 199
Failed: 1
Accuracy: 99.50%

❌ 失败用例列表:
{'id': 'multi_turn_base_154', 'error': 'list index out of range'}
